In [ ]:
## 06.Design Pattern with First-Class Functions ##

In [ ]:
# 1
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [ ]:
park = Customer('Park', 100)
park

Customer(name='Park', fidelity=100)

In [ ]:
class LineItem:
  """구매할 물품/갯수 생성해서 총 가격을 반환"""
  def __init__(self, product, quantity, price):
      self.product = product
      self.quantity = quantity
      self.price = price

  def total(self):
        return self.price * self.quantity

In [ ]:
class Order:
  def __init__(self, customer, cart, promotion=None):
      self.customer = customer
      self.cart = list(cart)
      self.promotion = promotion

  def total(self):
      if not hasattr(self, '__total'):
          self.__total = sum(item.total() for item in self.cart)
      return self.__total

  def due(self):
      if self.promotion is None:
          discount = 0
      else:
          discount = self.promotion.discount(self)
      return self.total() - discount

  def __repr__(self):
      fmt = '<Order total: {:.2f} due: {:.2f}>'
      return fmt.format(self.total(), self.due())

In [ ]:
class Promotion(ABC):
  """할인 혜택 클래스들의 형태를 선언"""

  @abstractmethod
  def discount(self, order):
    """할인액을 구체적인 숫자로 변환"""
    pass

In [ ]:
class FidelityPromo(Promotion):
  """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""

  def discount(self, order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""

    def discount(self, order):
      discount = 0
      for item in order.cart:
        if item.quantity >= 20:
          discount += item.total() * 0.1
      return discount

class LargeOrderPromo(Promotion):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""

    def discount(self, order):
      distinct_items = {item.product for item in order.cart}
      if len(distinct_items) >= 10:
        return order.total() * 0.07
      return 0

In [ ]:
# 2
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [
  LineItem('banana', 4, .5),
  LineItem('apple', 10, 1.5),
  LineItem('watermellon', 5, 5.0)
]

In [ ]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [ ]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [ ]:
banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]

In [ ]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [ ]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [ ]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [ ]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [ ]:
# 3
class Order:
  def __init__(self, customer, cart, promotion=None):
      self.customer = customer
      self.cart = list(cart)
      self.promotion = promotion

  def total(self):
      if not hasattr(self, '__total'):
          self.__total = sum(item.total() for item in self.cart)
      return self.__total

  def due(self):
      if self.promotion is None:
          discount = 0
      else:
          discount = self.promotion(self)
      return self.total() - discount

  def __repr__(self):
      fmt = '<Order total: {:.2f} due: {:.2f}>'
      return fmt.format(self.total(), self.due())

In [ ]:
def fidelity_promo(order):
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount

def large_order_promo(order):
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return 0

In [ ]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [ ]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [ ]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [ ]:
Order(ann, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [ ]:
# 4
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
  return max(promo(order) for promo in promos)

In [ ]:
# 5
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [ ]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [ ]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [ ]:
# 6
promos = [globals()[name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo']

promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [ ]:
## 07.Function Decorators and Closures ##

In [1]:
# 1
def deco(func):
  def inner():
    print('running inner()')
  return inner

In [2]:
@deco
def target():
  print('running target()')

target()

running inner()


In [3]:
target

<function __main__.deco.<locals>.inner()>

In [4]:
# 2
registry = []

def register(func):
  print('running register(%s)' % func)
  registry.append(func)
  return func

@register
def f1():
  print('running f1()')

@register
def f2():
  print('running f2()')

def f3():
  print('running f3()')

def main():
  print('running main()')
  print('registry ->', registry)
  f1()
  f2()
  f3()

if __name__ == '__main__':
  main()

running register(<function f1 at 0x7c13bc22f880>)
running register(<function f2 at 0x7c13bdac1900>)
running main()
registry -> [<function f1 at 0x7c13bc22f880>, <function f2 at 0x7c13bdac1900>]
running f1()
running f2()
running f3()


In [5]:
# 3
promos = []

def promotion(promo_func):
  promos.append(promo_func)
  return promo_func

@promotion
def fidelity_promo(order):
  """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promotion
def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
  return max(promo(order) for promo in promos)

In [6]:
# 4
def f1(a):
  print(a)
  print(b)

f1(3)

3


NameError: name 'b' is not defined

In [7]:
b=6
f1(3)

3
6


In [8]:
# 5
b = 6
def f2(a):
  print(a)
  print(b)
  b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [10]:
# 6
def f3(a):
  global b
  print(a)
  print(b)
  b = 9

f3(3)

3
6


In [11]:
b

9

In [12]:
f3(3)

3
9


In [13]:
b=30
f3(3)

3
30


In [14]:
# 7
class Averager():
  def __init__(self):
    self.series = []

  def __call__(self, new_value):
    self.series.append(new_value)
    total = sum(self.series)
    return total/len(self.series)

In [15]:
avg = Averager()
avg(10)

10.0

In [16]:
avg(11)

10.5

In [17]:
avg(12)

11.0

In [18]:
# 8
def make_averager():
  series = []

  def averager(new_value):
    series.append(new_value)
    total = sum(series)
    return total/len(series)

  return averager

In [19]:
avg = make_averager()
avg(10)

10.0

In [20]:
avg(11)

10.5

In [21]:
avg(12)

11.0

In [22]:
# 9
avg.__code__.co_varnames

('new_value', 'total')

In [23]:
avg.__code__.co_freevars

('series',)

In [24]:
avg.__closure__

(<cell at 0x7c137fcb9bd0: list object at 0x7c137f618940>,)

In [25]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [26]:
# 10
def make_averager():
  count = 0
  total = 0

  def averager(new_value):
    count += 1
    total += new_value
    return total / count

  return averager

In [27]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [28]:
# 11
def make_averager():
  count = 0
  total = 0

  def averager(new_value):
    nonlocal count, total
    count += 1
    total += new_value
    return total / count

  return averager

In [29]:
avg = make_averager()
avg(10)

10.0

In [32]:
# 12
import time

def clock(func):
  def clocked(*args):
    t0 = time.perf_counter()
    result = func(*args)
    elapsed = time.perf_counter() - t0
    name = func.__name__
    arg_str = ', '.join(repr(arg) for arg in args)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [33]:
# 13
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
  print('*' * 40, 'Calling snooze(.123)')
  snooze(.123)
  print('*' * 40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12321194s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000075s] factorial(1) -> 1
[0.00003740s] factorial(2) -> 2
[0.00006147s] factorial(3) -> 6
[0.00011757s] factorial(4) -> 24
[0.00014566s] factorial(5) -> 120
[0.00016973s] factorial(6) -> 720
6! = 720


In [34]:
# 14
import time
import functools

def clock(func):
  @functools.wraps(func)
  def clocked(*args, **kwargs):
    t0 = time.time()
    result = func(*args, **kwargs)
    elapsed = time.time() - t0
    arg_lst = []
    name = func.__name__
    if args:
      arg_lst.append(', '.join(repr(arg) for arg in args))
    if kwargs:
      pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
      arg_lst.append(', '.join(pairs))
    arg_str = ', '.join(arg_lst)
    print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [35]:
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
  print('*'*40, 'Calling snooze(.123)')
  snooze(.123)
  print('*'*40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12323737s] snooze(0.123) -> None 
**************************************** Calling factorial(6)
[0.00000095s] factorial(1) -> 1 
[0.00003791s] factorial(2) -> 2 
[0.00006127s] factorial(3) -> 6 
[0.00008178s] factorial(4) -> 24 
[0.00011635s] factorial(5) -> 120 
[0.00014806s] factorial(6) -> 720 
6! = 720
